In [ ]:
import numpy as np
from numpy import random
from numpy.random import uniform
from qiskit import  *
from qiskit import QuantumCircuit
from qiskit import IBMQ, Aer
from random import sample
from qiskit.quantum_info import Clifford
from qiskit import Aer, transpile
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.providers.aer import AerSimulator
from scipy.optimize import curve_fit
from numpy.linalg import inv, det
from scipy.optimize import fsolve
import qiskit as qiskit
from qiskit.circuit.library import SwapGate
from qiskit.circuit.library import YGate
from time import time
import time
from scipy.stats import rv_continuous
import random 
import math
from sympy import *
#import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# define the circuit 
class cir_ld(QuantumCircuit):

    # # define self of circuit
    # def __init__(self):
    #     return self

    # define 1-qubit clifford gate set: {'I', 'H', 'S'}
    def clifford_single_gate(self,qbi,gatei):  
       # if gatei=='i':
         #   self.id(qbi)
        if gatei=='h':
            self.h(qbi)
        elif gatei=='s':
            self.s(qbi)
    
    # define 1-qubit gate_universal: {'SX','SY','SP'}
    def universal_single_gate(self,qbi,gatei):
        p = 0.25
        t = 0.5
        sp = qiskit.quantum_info.Operator([
        [np.exp(1j*np.pi*t/2) * np.cos(np.pi * t/2), -1j*np.exp(1j*np.pi * (t/2 - p)) * np.sin(np.pi*t/2)],
        [-1j*np.exp(1j*np.pi * (t/2 + p)) * np.sin(np.pi*t/2), np.exp(1j*np.pi*t/2) * np.cos(np.pi * t/2)] ])
        if gatei=='sx':
            self.sx(qbi)
        elif gatei=='sy':
            self.sy(qbi)
        elif gatei=='sp':
            self.unitary(sp, [qbi])
    
    
    # define paulis
    def paulis(self,qbi,gi):
        if gi=='x':
            self.x(qbi)
        elif gi=='y':
            self.y(qbi) 
        elif gi=='z':
            self.z(qbi)
        elif gi=='i':
            self.id(qbi)
    
    # define composation fo circuit
    def cir_com(self,circ):
       self.compose(circ)

def clifford_single_gate_ext(cirs,qbi,gatei):  
        #if gatei=='i':
            #cirs.id(qbi)
        if gatei=='h':
            cirs.h(qbi)
        elif gatei=='s':
            cirs.s(qbi)
        return cirs

# define 1-qubit cli-gate layer 
def q1_cligate(nq):
    gi_index = []
    q1cli = ['s','h']
    # print (paulis,prob_sel)
    prob = (1/2) * np.ones(2)
    gi_index = np.random.choice(q1cli, size=1, replace=True, p = prob )[0]  
    return  gi_index

# define pauli layer 
def q1_pauli(nq):
    pi_index = []
    paulis = ['i','x','y','z']
    prob = (1/4) * np.ones(4)
    pi_index = np.random.choice(paulis, size=1, replace=True, p =  prob)[0]
    return   pi_index

# define cnot layer
def q2_cnot(cnotden,event_sel,width,length,depth): # prob is the density of cnot gate 
    nq = width * length
    ncot_tot = np.int(nq * depth * cnotden)
    sumcnot = 0
    q1cli_set = []
    q2cnot_set = []
    for i in range (depth):
        q2cnot_index = []
        if nq==4 or nq==9:
          #  ncnot = random.randint(1, 1) # 每层 num_cnot gate 随机
           ncnot =1
        # ncnot = np.int(nq * cnotden) # 每层 num_cnot gate 固定
        if nq ==16 or nq==25:
          #  ncnot = random.randint(1,1) # ncnot = random.randint(1,(nq-1)/2)--nq=9 有错！暂时
            ncnot = 1
            #ncnot = width
        #print (ncnot)
        tot_conn = connectivity(width,length,event_sel, ncnot) # 输出没有问题
        #print (tot_conn)
        index_tot_conn = np.arange(len(tot_conn))
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #print (tot_conn,ncnot) 
        q2cnot = np.random.choice(index_tot_conn,size=ncnot,replace=False)# !!!!!!!!!!!!!!!!!!!!!!!!!

        for j in q2cnot:     
            q2cnot_index.append(tot_conn[j]) # qubit indexes of cnot gate
        q2cnot_set.append(q2cnot_index)
        # the rest qubits which perform 1-qubit gate   
        q2cnot_re =[int(x) for item in ( np.array(q2cnot_index).reshape(ncnot*2,1).tolist()) for x in item]
        q1cli_set.append(list(set(np.arange(0,width*length)).difference(set(q2cnot_re))))
        sumcnot = sumcnot + ncnot # stop if sum of cnot equals ncot_tot
        if sumcnot == ncot_tot:
            break
        # 分开输出吧！！！
    return [q2cnot_set,q1cli_set]


# define connectivity (to give all possible connectivity) # 连接性的这里不应该混进来采样概率
def connectivity(width,length,event_sel,tot_conn):
    # {4 qubits,9 qubits,16 qubits, 25 qubits...}
    nq = length * width
    cnot_index = []

    # event=1: 水平方向cnot; 
    if event_sel=='s2': 
        for i in range (nq):
            if (i+1)%width!=0:
                cnot_index.append([i,i+1])

    # event=2: 竖直方向cnot       
    if event_sel=='s3': 
        for i in range (nq):
            if i+width<=nq-1:
                cnot_index.append([i,i+width]) 

    # to randomly choose qubits to cnot
    nrand = 1000* nq 
    cnot_index_sel = random.choices(cnot_index, weights=None, cum_weights=None, k=1)
    for i in  range (nrand):
        cnot_sel = random.choices(cnot_index, weights=None, cum_weights=None, k=1)
        if len(set([int(x) for item in (cnot_sel ) for x in item]) & set([int(x) for item in (cnot_index_sel) for x in item]))==0:
            cnot_index_sel.append(cnot_sel[0]) 
        if len(cnot_index_sel)==tot_conn:
            break
    return cnot_index_sel

# to generate random mirror circuit
def mir_cir_gene(dep,nq,width,length,event_sel,prob):
    circ = cir_ld(nq)
    circ_whole = cir_ld(nq)
    circ_right = cir_ld(nq)
    for qi in range (nq):
        circ.clifford_single_gate(qi,q1_cligate(nq))
    circ_cli_inverse = circ.inverse()

    # 1-qubit gate only
    if event_sel == 's1':
        circ_sgate_set = []
        # the left part
        for i in range (dep):
            circ_sgate = QuantumCircuit(nq)
            for j in range (nq):  
                q1_cli = q1_cligate(nq)
                circ.paulis(j,q1_pauli(nq))
                circ.clifford_single_gate(j,q1_cli)
                cli_s_re = clifford_single_gate_ext(circ_sgate,j,q1_cli)
            circ_sgate_set.append( cli_s_re)
        # print (circ)
        #!!!!!!!!!!!!!这里有问题啊！估计之前结果一直不太对，应该就是这里不对！！明天继续检查吧！！ mirror照镜子的第一层-不对，为什么其他的对呢
    
        # central pauli
        for qi in range (nq):
            circ.paulis(qi,q1_pauli(nq))
        # print (circ)
        
        # the right part
        for i in range (dep):
            circ_right =circ_right.compose(circ_sgate_set[dep-1-i].inverse())
            for j in range (nq):
                circ_right.paulis(j,q1_pauli(nq))
        # print (circ_right)
     
     # 2-qubit cnot layer
    if event_sel =='s2'or event_sel =='s3':
        circ_cnot_ori = q2_cnot(prob,event_sel,width,length,dep)
        # print (circ_cnot_ori)
        
        # the left part
        circ_cnot_copy_set = []
        for i in range (dep):
            circ_cnot_copy = QuantumCircuit(nq)
            cli_cn_sre =  QuantumCircuit(nq)
            for qi in range (nq):
                circ.paulis(qi,q1_pauli(nq))
        
            for e in circ_cnot_ori[0][i]: # cnot gate
                circ.cnot(e[0],e[1])
                circ_cnot_copy.cnot(e[0],e[1])
            # print (circ_cnot_copy) 
            
            for f in circ_cnot_ori[1][i]: # 1-qubit gate
                cliqrn = q1_cligate(len(circ_cnot_ori[1][i]))
                circ.clifford_single_gate(f,cliqrn)
                cli_cn_sre = clifford_single_gate_ext(circ_cnot_copy,f,cliqrn)   
            circ_cnot_copy_set.append(cli_cn_sre)    

        # the central pauli part
        for qi in range (nq):
            circ.paulis(qi,q1_pauli(nq))

        # the right part
        for i in range (dep):
            circ_right = circ_right.compose(circ_cnot_copy_set[dep-1-i].inverse())
            for qi in range (nq):
                circ_right.paulis(qi,q1_pauli(nq))           
    # the whole circuit
    circ_whole =  (circ.compose(circ_right)).compose(circ_cli_inverse)   
    circ_whole.measure_all()
    return   circ_whole 

# define noise model
def noise_model_building(circ,err_choice,err,errq1):
    noise_model=NoiseModel()

    if err_choice == '00': # pauli_cnot only
        q1pauli = pauli_error([('X',err), ('I', 1 - err)])
        q2pauli = q1pauli.tensor(q1pauli)
        noise_model.add_all_qubit_quantum_error(q2pauli, 'cx')

    if err_choice == '01': # pauli_cnot & 1-qubit
        q1pauli_cnot = pauli_error([('X',err),('I',1-err)])
        q1pauli_q1 = pauli_error([('X',errq1), ('I', 1 - errq1)])
        err_rp = 0.0001
        qrp_pauli = pauli_error([('X',err_rp),('I',1-err_rp)])
        noise_model.add_all_qubit_quantum_error(q1pauli_q1,  ['s','h'])
        noise_model.add_all_qubit_quantum_error(qrp_pauli,  ['id','x','y','z'])
        q2pauli = q1pauli_cnot.tensor(q1pauli_cnot)
        noise_model.add_all_qubit_quantum_error(q2pauli, 'cx')
        
    if err_choice == '10': # depchannel_cnot only
        q2depchan = depolarizing_error(err, 2)
        noise_model.add_all_qubit_quantum_error(q2depchan, 'cx')

    if err_choice == '11': # depchannel_cnot & 1-qubit
        q1depchan = depolarizing_error(errq1, 1)
        q2depchan = depolarizing_error(err, 2)
        noise_model.add_all_qubit_quantum_error(q1depchan, ['s','h'])
        noise_model.add_all_qubit_quantum_error(q2depchan, 'cx')
        err_rp = 0.0001
        rpdepchan = depolarizing_error(err_rp, 1)
        noise_model.add_all_qubit_quantum_error(rpdepchan, ['id','x','y','z'])
        
        
    
    # if err_choice =='20' # 
    return noise_model

# define random compling
# def random_compling(circ):
# return circ


# define ideal aer simulation
def cir_ideal_simulation(circ):
    res_ideal =  AerSimulator().run(circ,shots=8192).result()
    counts_ideal = res_ideal.get_counts(circ)
    return counts_ideal

# define noisy aer simulation
def cir_noise_simulation(cir,ns_model):
    #trcir = transpile(cir,AerSimulator(noise_model=ns_model))
    shot = 8192
    res_noise = execute(cir, backend=Aer.get_backend('qasm_simulator'),\
            noise_model=ns_model, shots=shot,optimization_level=0).result()
    counts_noise = res_noise.get_counts(cir)
    return counts_noise

# to estimate the effective polarization
def eff_depo(counts_id,counts_err,nq):
    def hamming_distance(string1, string2): 
        distance = 0
        for i in range(len(string1)):
            if string1[i] != string2[i]:
                distance = distance + 1
        return distance
    bit_ideal = list(counts_id.keys())[0]
    bit_exp = list(counts_err.keys())
    # print (bit_exp)
    probsum = 0
    shot = 8192
    #print (bit_exp[0],bit_ideal)
    for i in range (len(bit_exp)):
        hamdis = hamming_distance(bit_ideal,bit_exp[i])
        probsum = probsum + ((-(1/2))**hamdis) * (counts_err[bit_exp[i]]/shot)
        #print(bit_ideal,bit_exp[i],hamdis)

    eff_depolarization = (4**(nq)/(4**(nq)-1)) * probsum - (1/(4**(nq)-1))
    return eff_depolarization

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   
# mirror circuit error rate simulation
# 2 events:{s1-single gate only;s2-cnot(v/h)}
# num_omg = 2
# v1 = [1/2,1/2]
# v2 = [1/8,7/8]
# prob_omg = [v1,v2]

# 3 events:{s1-single gate only;s2-cnot(v/h)}
num_omg = 3
v1 = [18/20,1/20,1/20] 
v2 = [1/4,1/4,1/2]
v3 = [1/4,1/2,1/4] 

#prob_omg = np.array([v1,v2,v3])
prob_omg = np.array([v3,v2,v1])

width = 2
length = 2
nq = width * length
depmax = 40 # depth of circuit
ncir = 50 # the repeated times for a dep-circuit 
#depth_list = np.arange (5,depmax+1,5)
depth_list = [5,10,15]
#depth_list = [2,4,6,8,10]
# event_set = ['s1','s2']
event_set = ['s1','s2','s3'] 
err_choice = '01'
q1s2_err = 0.002
q1s3_err = 0.004
q1s_err = 0.003


start_dep = time.perf_counter()
den_cnot = 1
num_run = 1
cir_mirror_set = []
numest = 10 # num of estimation
err_epcelon1 = []
err_epcelon2 = []
err_epcelon3 = []

err_cnot1 = []
err_cnot2 = []
err_cnot3 = []
for s in range (numest):
  err_omg = []
  for i in range (num_omg):
    # print (prob_omg[i])
   # print (event_sel)
    depo_eff_dep_mean = []
    for d in depth_list: 
        
        depo_eff_dep = []
        for j in range (ncir): 
                event_sel = np.random.choice(event_set, size=1, replace=True, p = prob_omg[i])[0] # randomly choose event
                cir_mirror = cir_ld(nq)
                cir_mirror = mir_cir_gene(d,nq,width,length,event_sel,den_cnot)
                cir_mirror_set.append(cir_mirror)
                cir_mirror_copy = cir_mirror.copy()
                # cir_mirror_copy = cir_mirror.copy()
                # print (event_sel)
             
                counts_ideal = cir_ideal_simulation(cir_mirror) # ideal counts
                counts_error = counts_ideal
                dep_run = []
                for k in range (num_run):
                    # noise model& # noisy counts   
                    if event_sel == 's1': 
                        if err_choice=='00' or err_choice=='10':
                            counts_error = counts_ideal
                        if err_choice=='01' or err_choice=='11':
                            ns_model = noise_model_building(cir_mirror_copy,err_choice,q1s2_err,q1s_err)
                            counts_error = cir_noise_simulation(cir_mirror_copy,ns_model)

                    if event_sel == 's2': 
                        ns_model = noise_model_building(cir_mirror_copy,err_choice,q1s2_err,q1s_err)
                        counts_error = cir_noise_simulation(cir_mirror_copy,ns_model)

                    if event_sel == 's3': 
                        ns_model = noise_model_building(cir_mirror_copy,err_choice,q1s3_err,q1s_err)
                        counts_error = cir_noise_simulation(cir_mirror_copy,ns_model)
                    dep_run.append(eff_depo(counts_ideal,counts_error,nq))
                depo_eff_dep.append(np.mean(dep_run))       
        depo_eff_dep_mean.append(np.mean(depo_eff_dep))
    
    # to fit error rate
    def func_fit(x,p,A):
        return A * (p**x)
    x1 =  [i*2 for i in depth_list]
    y1 = depo_eff_dep_mean
    p1_fit = curve_fit(func_fit, x1, y1)[0][0]
    err_omg.append((4**nq-1) * (1-p1_fit)/(4**nq))  


  # to estimate err_rate #
  #prob_omg_s = np.array([v3,v2,v1])

  eps_mir = inv(prob_omg)* (np.matrix(err_omg).T)
  err_epcelon1.append(eps_mir[0,0])
  err_epcelon2.append(eps_mir[1,0])
  err_epcelon3.append(eps_mir[2,0])


  if num_omg==2:
    def solve_function_mir(unsolved_value):
        eps_local,eps_s2 =unsolved_value[0],unsolved_value[1]
        return [1 - (1 - eps_local)**nq - eps_mir[0,0],
                1 - (((1- eps_local)**(nq-2))*(1 - eps_s2))-eps_mir[1,0]]
    err_cnot = fsolve(solve_function_mir,[0,0])
    err_cnot1.append(err_cnot[0])
    err_cnot2.append(err_cnot[1])
    err_cnot3.append(err_cnot[2])

  if num_omg==3:
    def solve_function_mir(unsolved_value):
        eps_local,eps_s2,eps_s3 =unsolved_value[0],unsolved_value[1],unsolved_value[2]
        return [1 - (1 - eps_local)**nq - eps_mir[0,0],
        1 - (((1- eps_local)**(nq-2))*(1 - eps_s2))-eps_mir[1,0],
        1 - (((1- eps_local)**(nq-2))*(1 - eps_s3))-eps_mir[2,0]]
    err_cnot = fsolve(solve_function_mir,[0,0,0])
    err_cnot1.append(err_cnot[0])
    err_cnot2.append(err_cnot[1])
    err_cnot3.append(err_cnot[2])
    
# output
print ('err_epcircuit1 = ',np.mean(err_epcelon1)) 
print ('err_epcircuit2 = ',np.mean(err_epcelon2)) 
print ('err_epcircuit3 = ',np.mean(err_epcelon3)) 

# print (err_cnot)
print ('err_1qgate = ',np.mean(err_cnot1)) 
print ('err_cnots2 = ',np.mean(err_cnot2)) 
print ('err_cnots3 = ',np.mean(err_cnot3)) 
print ('elapsed =' ,time.perf_counter() - start_dep)
